In [1]:
import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer
beer_data = cPickle.load(open('../../beer_data/beer_vec_ds_df10.p', 'rb'))

Using TensorFlow backend.


[]


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

ds = pd.read_csv('../../beer_data/beer_ds.csv')

from ast import literal_eval as make_tuple
ds['bits'] = ds['bits'].map(lambda s : make_tuple(s))

aspect_columns = sorted(['review/appearance', 'review/taste', 'review/aroma', 'review/palate'])

train_idxs, val_idxs = train_test_split(ds.index, stratify=ds['bits'], train_size=0.9, random_state=1337)

/home/sarthak/anaconda2/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [3]:
train_X = beer_data.X[train_idxs]
X_tf = np.zeros((train_X.shape[0], beer_data.vocab_size))
for i in range(len(train_X)) :
    X_tf[i, train_X[i, :]] = 1.

X_tf = X_tf[:, 2:]
train_Xtf = X_tf

val_X = beer_data.X[val_idxs]
X_tf = np.zeros((val_X.shape[0], beer_data.vocab_size))
for i in range(len(val_X)) :
    X_tf[i, val_X[i, :]] = 1.

X_tf = X_tf[:, 2:]
val_Xtf = X_tf

In [4]:
idxs = val_idxs
H = {}
for i in range(4) :
    H[str(i)] = np.zeros((len(idxs), len(idxs)))
    a0 = set(ds[ds[aspect_columns[i]] == 0].index) & set(idxs)
    a1 = set(ds[ds[aspect_columns[i]] == 1].index) & set(idxs)
    a0 = map(lambda s : list(idxs).index(s), a0)
    a1 = map(lambda s : list(idxs).index(s), a1)
    for j in a0 :
        H[str(i)][j, a0] = 1
    for j in a1 :
        H[str(i)][j, a1] = 1

    H[str(i)][np.arange(len(idxs)), np.arange(len(idxs))] = 0

In [10]:
from keras.layers import Input, Dense, Lambda
from keras.models import Model
from keras import backend as K
from keras import objectives
from keras import optimizers

learning_rate = 5e-5
batch_size = 128
vocab_size = beer_data.vocab_size - 2
intermediate_dim = 500
latent_dim = 4
epochs = 1000
epsilon_std = 1.0
activation = 'tanh'
from sklearn.metrics import roc_auc_score

for n_top in range(1, 200, 5) :
    latent_dim = n_top
    x = Input(shape=(vocab_size,), name='x')
    h = Dense(intermediate_dim, activation=activation, name='h')(x)
    mu = Dense(latent_dim, name='mu')(h)
    log_sigma2 = Dense(latent_dim, name='l')(h)
    encoder = Model(x, mu)

    # reparameterized sampler for normal distributions
    def sample_norm(args):
        '''reparameterized sampling from normal distribution'''
        mu, log_var = args
        epsilon = K.random_normal(shape=(K.shape(mu)[0], latent_dim,), mean=0.)
        return mu + K.exp(0.5 * log_var) * epsilon

    # decoder / generative network
    z = Lambda(sample_norm, output_shape=(latent_dim,), name='z')([mu, log_sigma2])
    e = Dense(vocab_size, name='e')(z)

    def log_softmax(x, axis=None):
        x0 = x - K.max(x, axis=axis, keepdims=True)
        log_sum_exp_x0 = K.log(K.sum(K.exp(x0), axis=axis, keepdims=True))
        return x0 - log_sum_exp_x0

    def kl_loss(x, e): 
        return (- 0.5 * K.sum(1 + log_sigma2 - K.square(mu) - K.exp(log_sigma2), axis=-1))


    def cross_ent_loss(x, e): 
        return - K.sum(x * log_softmax(e, axis=-1), axis=-1) 


    def vae_loss(x, e):
        xent_loss = cross_ent_loss(x, e)
        kld = kl_loss(x, e)
        return xent_loss + kld


    opt = optimizers.adam(lr=learning_rate)
    vae = Model(x, e)
    vae.compile(optimizer=opt, 
                loss=vae_loss)
    from keras import callbacks
    patience = 0
    earlyStopping = callbacks.EarlyStopping(monitor='val_loss', patience=patience, verbose=1, mode='min')

    vae.fit(train_Xtf,  
            train_Xtf, 
            shuffle=True,
            epochs=10,
            batch_size=batch_size,
            verbose=0, 
            callbacks=[earlyStopping], 
            validation_split=0.1)
    embedds = encoder.predict(val_Xtf)
    from sklearn.preprocessing import normalize
    embedds_n = normalize(embedds, 'l2')
    scores = np.dot(embedds_n, embedds_n.T)
    nb_studies = len(val_idxs)
    scores[np.arange(nb_studies), np.arange(nb_studies)] = -1000
    aucs = [0] * nb_studies
    for i in range(nb_studies) :
        aucs[i] = roc_auc_score(H['0'][i], scores[i])
    print n_top, np.mean(aucs)

1 0.500170493183
6 0.5831631426
11 0.654411021299
16 0.632510741257
21 0.665045907498
26 0.700779968083
31 0.688440305402
36 0.689175010013
41 0.707412652339
46 0.702140864486
51 0.704337578871
56 0.708874523588
61 0.698482832347
66 0.701969316004
71 0.706314184499
76 0.712376043307
81 0.706971200091
86 0.707239246642
91 0.703389710955
96 0.708253497004
101 0.710427913085
106 0.707307906607
111 0.710253068616
116 0.710623215004
121 0.718300717019
126 0.717827256605
131 0.711738942091
136 0.713365719461
141 0.71196521616
146 0.714044095257
151 0.710920213292
156 0.710111289099
161 0.708467073387
166 0.711255949706
171 0.713655512221
176 0.713832026132
181 0.708481051749
186 0.711625241759
191 0.712755709526
196 0.710443357611
